In [4]:
print('OK')

OK


In [5]:
%pwd

'c:\\End-to-End-Medical-Chatbot\\research'

In [7]:
import os
os.chdir("../")

In [8]:
%pwd

'c:\\End-to-End-Medical-Chatbot'

In [16]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [17]:
#Extract Data from PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [20]:
extracted_data=load_pdf_file(data='C:\End-to-End-Medical-Chatbot\Data')

In [12]:
# extracted_data

In [21]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 39994


In [16]:
# text_chunks

In [ ]:
# from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
#Downloading the Embeddings from Hugging Face
# def download_hugging_face_embeddings():
#     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#     return embeddings

In [42]:
# embeddings=download_hugging_face_embeddings()

In [13]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [24]:
from langchain.embeddings import HuggingFaceEmbeddings

In [25]:
#Downloading the Embeddings from Hugging Face
def download_hugging_face_embeddings():
     embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
     return embeddings

In [26]:
embeddings= download_hugging_face_embeddings()

C:\Users\nikita kamnani\AppData\Local\Temp\ipykernel_23376\620568460.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [16]:
query_result = embeddings.embed_query("Hello World")
print("Length",len(query_result))

Length 384


In [14]:
# query_result

In [27]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
# import os

In [32]:
# print(os.getenv("PINECONE_API_KEY"))
# print(os.getenv("COHERE_API_KEY"))


In [28]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")

In [29]:
import os
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec, PineconeApiException

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define the index name
index_name = "medicalbot"

# List existing indexes
existing_indexes = pc.list_indexes()

# Check if the index already exists
if index_name in existing_indexes:
    print(f"Index '{index_name}' already exists. Skipping creation.")
else:
    try:
        print(f"Creating index: {index_name}")
        pc.create_index(
            name=index_name,
            dimension=384,  # Replace with your model dimensions
            metric="cosine",  # Replace with your model metric
            spec=ServerlessSpec(cloud="aws", region="us-east-1")  # Specify your cloud and region
        )
        print(f"Index '{index_name}' created successfully.")
    except PineconeApiException as e:
        print(f"Error creating index: {e}")

Creating index: medicalbot
Error creating index: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '50f7c25045585595602b8a0edfd8cd35', 'Date': 'Tue, 18 Feb 2025 10:58:47 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}



In [14]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["COHERE_API_KEY"]=COHERE_API_KEY

In [2]:
# pip install langchain-pinecone


In [30]:
#Embed each chunk and upsert the embeddings into your Pinecone Index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [31]:
# Load Existing Index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecone Index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
docsearch

In [33]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [34]:
retrieved_docs=retriever.invoke("What is Acne?")

In [35]:
retrieved_docs

[Document(id='0990dd7f-f77d-4b4a-a83a-bc08380afb60', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'C:\\End-to-End-Medical-Chatbot\\Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='cff8bb46-097c-46b4-83f0-96019496b801', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 55.0, 'page_label': '26', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'C:\\End-to-End-Medical-Chatbot\\Data\\The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_.pdf', 'total_pages': 4505.0}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF ME

In [36]:
# As i want defination so process is 
import os
from langchain.llms import Cohere

# Set API Key in environment variables
os.environ["COHERE_API_KEY"] = "dPlXVSswiGgW9sudLfu1z8lPAu68oGV6OMMXDSHx"

# Initialize the Cohere LLM
llm = Cohere(temperature=0.4, max_tokens=500)

# Example usage
# response = llm.invoke("What is Acne?")
# print(response)


In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know"
    "use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [38]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain= create_retrieval_chain(retriever,question_answer_chain)

In [39]:
response = rag_chain.invoke({"input":"What is Acne?"})
print(response["answer"])

 Acne is a chronic skin disease involving blockage and inflammation of the sebaceous glands and hair follicles. It is characterized by whiteheads, blackheads, and inflamed lesions. In mild cases, acne usually resolves itself, but it can lead to scarring in more severe cases. 
